# County Health Rankings Data from Tableau Public

Exploring data from: 

https://public.tableau.com/en-us/s/resources?qt-overview_resources=1

https://public.tableau.com/s/sites/default/files/media/County_Health_Rankings.csv

Looks like the original sources is here: https://www.countyhealthrankings.org/app/

And, maybe...

https://www.nber.org/workfamily/
https://www.nber.org/workfamily/WorkFamily_rev.dta

**Imports**  
Import libraries, write settings, and bring in files here.

In [3]:
# Data manipulation
import pandas as pd
import numpy as np

# Options for pandas
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

## Exploring Data

In [4]:
CH_ranking = pd.read_csv('../../data/01_raw/County_Health_Rankings.csv', dtype={'State code': 'str', 'County code': 'str', 'fipscode': 'str'})

In [45]:
CH_ranking.head()

,State,County,State code,County code,Year span,Measure name,Measure id,Numerator,Denominator,Raw value,Confidence Interval Lower Bound,Confidence Interval Upper Bound,Data Release Year,fipscode
0,US,United States,0,0,2003-2005,Violent crime rate,43.0,1328750.667,274877117.0,483.398066,NaN,NaN,NaN,0
1,US,United States,0,0,2004-2006,Violent crime rate,43.0,1340928.667,277612778.5,483.021233,NaN,NaN,NaN,0
2,US,United States,0,0,2005-2007,Violent crime rate,43.0,1355853.167,280407694.7,483.529230,NaN,NaN,2010.0,0
3,US,United States,0,0,2006-2008,Violent crime rate,43.0,1366928.333,287614567.7,475.263942,NaN,NaN,2011.0,0
4,US,United States,0,0,2007-2009,Violent crime rate,43.0,1339439.333,292576281.2,457.808585,NaN,NaN,2012.0,0


In [9]:
CH_ranking.columns

Index(['State', 'County', 'State code', 'County code', 'Year span',
       'Measure name', 'Measure id', 'Numerator', 'Denominator', 'Raw value',
       'Confidence Interval Lower Bound', 'Confidence Interval Upper Bound',
       'Data Release Year', 'fipscode'],
      dtype='object')

In [11]:
CH_ranking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303864 entries, 0 to 303863
Data columns (total 14 columns):
State                              303858 non-null object
County                             303858 non-null object
State code                         303860 non-null float64
County code                        303860 non-null float64
Year span                          303390 non-null object
Measure name                       303390 non-null object
Measure id                         303390 non-null float64
Numerator                          214076 non-null float64
Denominator                        184779 non-null float64
Raw value                          289956 non-null float64
Confidence Interval Lower Bound    189412 non-null float64
Confidence Interval Upper Bound    189412 non-null float64
Data Release Year                  150129 non-null float64
fipscode                           294283 non-null float64
dtypes: float64(10), object(4)
memory usage: 32.5+ MB


Need to clean up State and County codes to make them strings with the appropriate number of digits. 

In [10]:
CH_ranking['Measure name'].value_counts()

Children in poverty                41499
Premature Death                    38312
Unemployment                       35202
Daily fine particulate matter      28728
Adult obesity                      22351
Violent crime rate                 22351
Physical inactivity                22351
Preventable hospital stays         19159
Mammography screening              19159
Diabetic screening                 19159
Uninsured                          19156
Sexually transmitted infections    15963
Name: Measure name, dtype: int64

In [46]:
CH_ranking[CH_ranking['State code'] == '24'].County.value_counts()

Prince George's County    95
Frederick County          95
Washington County         95
Maryland                  95
Dorchester County         95
Garrett County            95
Montgomery County         95
Baltimore County          95
Calvert County            95
St. Mary's County         95
Allegany County           95
Anne Arundel County       95
Harford County            95
Wicomico County           95
Kent County               95
Charles County            95
Somerset County           95
Baltimore city            95
Talbot County             95
Howard County             95
Worcester County          95
Carroll County            95
Queen Anne's County       95
Caroline County           95
Cecil County              95
Name: County, dtype: int64

In [14]:
maryland = CH_ranking[CH_ranking['State code'] == '24']

**Summary of Data set**

Spans years: 1997 - 2012 (looks like more is available on their website: https://www.countyhealthrankings.org/)

These are the measures being reported in this set: 
- Children in poverty                41499
- Premature Death                    38312
- Unemployment                       35202
- Daily fine particulate matter      28728
- Adult obesity                      22351
- Violent crime rate                 22351
- Physical inactivity                22351
- Preventable hospital stays         19159
- Mammography screening              19159
- Diabetic screening                 19159
- Uninsured                          19156
- Sexually transmitted infections    15963

Need to change State Code, County Code and Fips Code to strings with padded zeros. 

### Cleaning up INT columns that should be STRING

turns out, I can just read the columns in with a specific data type and forego this business here. 

In [11]:
## Changing County code to string with 3-characters
CH_ranking['County code'] = CH_ranking['County code'].apply(lambda x: str(x).zfill(3))


In [12]:
## Changing State code to string with 2-characters

CH_ranking['State code'] = CH_ranking['State code'].apply(lambda x: str(x).zfill(2))


In [13]:
## Changing FIPS code to string with 5-characters
CH_ranking['fipscode'] = CH_ranking['fipscode'].apply(lambda x: str(x).zfill(5))


In [54]:
maryland['Confidence Interval Lower Bound'].isna().sum()/len(maryland)

0.34821052631578947

In [57]:
maryland['Measure name'].value_counts()
maryland['Year span'].value_counts()
maryland.info()

Children in poverty                325
Premature Death                    300
Unemployment                       275
Daily fine particulate matter      225
Violent crime rate                 175
Physical inactivity                175
Adult obesity                      175
Uninsured                          150
Preventable hospital stays         150
Diabetic screening                 150
Mammography screening              150
Sexually transmitted infections    125
Name: Measure name, dtype: int64

2008         250
2010         250
2009         250
2011         200
2007         175
2006         150
2004         150
2005         125
2003-2005    125
2003         100
2006-2007     75
2002          50
2007-2009     50
2004-2006     50
2008-2010     50
2012          50
2005-2007     50
2006-2008     50
2000-2002     25
2002-2004     25
2001-2003     25
1997-1999     25
1998-2000     25
1999-2001     25
2009-2011     25
Name: Year span, dtype: int64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2375 entries, 8484 to 286259
Data columns (total 14 columns):
State                              2375 non-null object
County                             2375 non-null object
State code                         2375 non-null object
County code                        2375 non-null object
Year span                          2375 non-null object
Measure name                       2375 non-null object
Measure id                         2375 non-null float64
Numerator                          1700 non-null float64
Denominator                        1512 non-null float64
Raw value                          2362 non-null float64
Confidence Interval Lower Bound    1548 non-null float64
Confidence Interval Upper Bound    1548 non-null float64
Data Release Year                  1175 non-null float64
fipscode                           2300 non-null object
dtypes: float64(7), object(7)
memory usage: 278.3+ KB


In [15]:
maryland['County code'].unique()

array(['000', '001', '003', '005', '009', '011', '013', '015', '017',
       '019', '021', '023', '025', '027', '029', '031', '033', '035',
       '037', '039', '041', '043', '045', '047', '510'], dtype=object)

In [67]:
maryland.loc[(maryland['County']== "Prince George's County"), 'Year span'].value_counts()

2009         10
2008         10
2010         10
2011          8
2007          7
2004          6
2006          6
2005          5
2003-2005     5
2003          4
2006-2007     3
2008-2010     2
2006-2008     2
2012          2
2005-2007     2
2004-2006     2
2007-2009     2
2002          2
1999-2001     1
2001-2003     1
1997-1999     1
2000-2002     1
2009-2011     1
1998-2000     1
2002-2004     1
Name: Year span, dtype: int64

## Analysis/Modeling
Need to think about how to organize the data for plotting and calculations. 

I could make each county it's own data frame. Where I could have years as the column and the factors/outcomes as the rows. 

## Results
Show graphs and stats here

## Conclusions and Next Steps
Summarize findings here